# Zillow Scrapper
## Julie Huang

In [31]:
## Import beautiful soup to scrape data from Zillow
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [52]:
class Property(object):
    """A property listed on Zillow


    Attributes:
        address: The address of the property as a string.
        bed: An integer representing the number of bedrooms the property has.
        bath: A float representing the no. of bathrooms the property has
        size: An interger representing the sqft of the property
        price: An interger representing the price of the property
    """


    def __init__(self):
        self.address = ""
        self.bed = 0.0
        self.bath = 0.0
        self.size = 0.0
        self.price = 0.0
        
    def price_per_sqft(self):
        """Return the sale price for this vehicle as a float amount."""
        if self.size != 0.0:
            pps = self.price/(self.size)
        else:
            pps = "Empty"
        return pps
    
    def price_per_bed(self):
        if self.bed != 0.0:
            ppb = self.price/self.bed
        else:
            ppb = "Empty"
        """Return the sale price for this vehicle as a float amount."""
        return ppb

In [54]:
zipcode = raw_input("Type a zipcode")
url= "https://www.zillow.com/homes/"+zipcode +"_rb/"
print url

Type a zipcode01375
https://www.zillow.com/homes/01375_rb/


In [55]:
r = requests.get(url)
#r.content

In [56]:
soup = BeautifulSoup(r.content)

In [57]:
print soup.prettify()

<!DOCTYPE html>
<html class="wf-loading no-js zsg-theme-modernized " itemscope="" itemtype="http://schema.org/Organization" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/product#">
 <head>
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="//www.zillowstatic.com" rel="dns-prefetch"/>
  <link href="//fonts.googleapis.com" rel="dns-prefetch"/>
  <link href="//photos.zillowstatic.com" rel="dns-prefetch"/>
  <meta charset="utf-8"/>
  <title>
   01375 Real Estate - 01375 Homes For Sale  | Zillow
  </title>
  <meta content="Zillow has 14 homes for sale in 01375. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>
  <meta content="Zillow, Inc." name="author"/>
  <meta content="Copyright (c) 2006-2017 Zillow, Inc." name="Copyright"/>
  <script>
   var UI_INIT_AT = Date.now ? Date.now() : +(

In [58]:
for item in soup.find_all("div", {'class':'zsg-photo-card-caption'}):
    #print item.contents[0].text #Apartment For Rent
    #print item.contents[1].text #1$1,585+2$2,145+ #$239,9002 bds · 1 ba · 738 sqft
    #print item.contents[2].text #18 hours ago·20 Park View Ter # 000, Malden, MA
    print item.contents[1].text #$1,500/mo
    #print item.contents[1].find_all("span", {'class':'interpunct'}) #1 bd · 1 ba · -- sqft

$295,0003 bds · 2 ba · 1,413 sqft
$1,200/mo1 bd · 1 ba · 800 sqft
$649,0004 bds · 5 ba · 4,000 sqft
$650/moStudio · 1 ba · -- sqft
$445,0004 bds · 3 ba · 3,092 sqft
$2,000/mo4 bds · 2 ba · 1,002 sqft
$319,9003 bds · 3 ba · 1,350 sqft
$305,0003 bds · 3 ba · 1,872 sqft
$299,0003 bds · 1 ba · 1,555 sqft
$88,9002 bds · 1 ba · 980 sqft
$90,0002 bds · 1 ba · 980 sqft
$349,0003 bds · 2 ba · 1,251 sqft
$59,0003.59 ac lot
Foreclosure Est. $245K · 4 bds · 3 ba · 2,448 sqft


In [59]:
Zillow = {}
id = 0
for item in soup.find_all("div", {'class':'zsg-photo-card-caption'}):
#    if item.get('class')==['zsg-photo-card-caption']:
        P = Property()
        for i in item.find_all("span"):
            if i.get('class')==['zsg-photo-card-price']:
                P.price = i.text
                P.price = float(i.text.replace("$","").replace("/mo","").replace(",","").replace("+",""))
                if P.price == 0.0:
                    P.price = NA
            else: 
                pass
            if i.get('class')==['zsg-photo-card-info'] and P.price: #P.price needs to exist to proceed on info and address
                P.info = i.text #info needs to be sliced
                P.bed = float(P.info.split()[0]) #This number can be 0
                P.bath = float(P.info.split()[3])
                P.size = float(P.info.split()[6].replace("--","0").replace("sqft","0").replace(",","").replace("+","")) #This number can be "0"
            if i.get('class')==['zsg-photo-card-address'] and P.price:
                P.address = i.text  
        print str(P.price)+" price\n"+ str(P.bed)+" bed\n"+ str(P.bath)+" bath\n"+ str(P.size)+" size\n"+str(P.address) +" address\n"+str(P.price_per_sqft())+" price_per_sqft\n\n\n"
        Zillow.update({id:P})
        id+=1
        
##info 1$1,585+2$2,145+
##adress 500 Broadway, Malden, MA

####We do not want it when price is empty

##price $1,700/mo
##info 2 bds · 1 ba · -- sqft
##adress Newman Rd, Malden, MA

####This is the type we want

295000.0 price
3.0 bed
2.0 bath
1413.0 size
30 Meadowbrook Dr, Sunderland, MA address
208.775654636 price_per_sqft



1200.0 price
1.0 bed
1.0 bath
800.0 size
313 Amherst Rd, Sunderland, MA address
1.5 price_per_sqft



649000.0 price
4.0 bed
5.0 bath
4000.0 size
145 Plumtree Rd, Sunderland, MA address
162.25 price_per_sqft





ValueError: could not convert string to float: Studio

In [60]:
for i in range(0,id):
    print str(Zillow[i].price) + '   ' + str(Zillow[i].bed) + '   '  + str(Zillow[i].size)+ '   ' + Zillow[i].address

295000.0   3.0   1413.0   30 Meadowbrook Dr, Sunderland, MA
1200.0   1.0   800.0   313 Amherst Rd, Sunderland, MA
649000.0   4.0   4000.0   145 Plumtree Rd, Sunderland, MA


In [ ]:
## Use the Zillow API, PyZillow to get real estate data